In [8]:
# 단층 퍼셉트론

import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
# XOR 문제에 해당되는 입력과 출력을 정의
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# 단층 퍼셉트론
linear = nn.Linear(2,1)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

# 0 또는 1을 예측하는 이진 분류 문제이므로 비용 함수로는 크로스엔트로피 함수를 사용
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    optimizer.zero_grad()
    y_pred = model(X)
    
    cost = criterion(y_pred, Y)
    cost.backward()
    
    optimizer.step()
    
    if step % 1000 == 0: # 1000번째 에포크마다 비용 출력
        print(step, cost.item())
        
# 더 이상 비용이 줄어들지 않음 -> 단층 퍼셉트론은 XOR문제를 해결할 수 없음

# 학습된 단층 퍼셉트론의 예측값 확인
with torch.no_grad():
    y_pred = model(X)
    predicted = (y_pred>0.5).float()
    accuracy = (predicted==Y).float().mean()
    print('모델의 출력값(Hypothesis): ', y_pred.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

0 0.7273974418640137
1000 0.6931471824645996
2000 0.6931471824645996
3000 0.6931471824645996
4000 0.6931471824645996
5000 0.6931471824645996
6000 0.6931471824645996
7000 0.6931471824645996
8000 0.6931471824645996
9000 0.6931471824645996
10000 0.6931471824645996
모델의 출력값(Hypothesis):  [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted):  [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


In [13]:
# 다층 퍼셉트론

import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
# XOR 문제에 해당되는 입력과 출력을 정의
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# 다층 퍼셉트론 (4층 퍼셉트론)
model = nn.Sequential(
    nn.Linear(2, 10),
    nn.Sigmoid(),
    
    nn.Linear(10,10),
    nn.Sigmoid(),
    
    nn.Linear(10, 10),
    nn.Sigmoid(),
    
    nn.Linear(10,1),
    nn.Sigmoid()
    ).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    y_pred = model(X)

    # 비용 함수
    cost = criterion(y_pred, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 1000 == 0:
        print(epoch, cost.item())
        
# 학습된 다층 퍼셉트론의 예측값 확인
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())    

0 0.6948983669281006
1000 0.6931380033493042
2000 0.6931172013282776
3000 0.6930763721466064
4000 0.6929103136062622
5000 0.6820822358131409
6000 0.001303164754062891
7000 0.000484021264128387
8000 0.00028917036252096295
9000 0.00020395874162204564
10000 0.0001566387654747814
모델의 출력값(Hypothesis):  [[1.11760564e-04]
 [9.99828696e-01]
 [9.99842167e-01]
 [1.85418889e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0


In [15]:
# 다층 퍼셉트론으로 손글씨 분류하기

%matplotlib inline
import matplotlib.pyplot as plt # 시각화를 위한 맷플롯립
from sklearn.datasets import load_digits
import torch
import torch.nn as nn
from torch import optim


digits = load_digits() # 1,979개의 이미지 데이터 로드

print(digits.images[0])  # 0을 흰색 도화지, 0보다 큰 숫자들을 검은색 점
print(digits.target[0])  # 실제 레이블
print(digits.data[0])  # 8 × 8 행렬을 전부 64차원의 벡터로 변환
X = digits.data # 이미지. 즉, 특성 행렬
Y = digits.target # 각 이미지에 대한 레이블

# 다층 퍼셉트론 분류기
model = nn.Sequential(
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16,10)
)

X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.int64)

loss = nn.CrossEntropyLoss()  # 소프트맥스 함수를 포함
optimizer = optim.Adam(model.parameters())
losses = []

for epoch in range(100):
    y_pred = model(X)
    cost = loss(y_pred, Y)
    
    optimizer.zero_grad()
    cost.backward()
    oprimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, 100, cost.item()
        ))
    
    lossed.append(cost.item())
    
plt.plot(losses)

ImportError: DLL load failed: 지정된 모듈을 찾을 수 없습니다.

In [22]:
# 다층 퍼셉트론으로 MNIST 분류하기

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, cache=True)
print(mnist.data[0])
print(mnist.target[0])

mnist.target.astype(np.int8)

X = mnist.data / 255  # 0-255값을 [0,1] 구간으로 정규화
y = mnist.target


import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=0)

x_train = torch.Tensor(X_train)
x_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

ds_train = TensorDataset(x_train, y_train)
ds_test = TensorDataset(x_test, y_test)

loader_train = DataLoader(ds_train, batch_size=64, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=64, shuffle=True)

model = nn.Sequential()
model.add_module('fc1', nn.Linear(28*28*1, 100))
model.add_module('relu1', nn.ReLU())
model.add_module('fc2', nn.Linear(100, 100))
model.add_module('relu2',nn.ReLU())
model.add_module('fc3',  nn.Linear(100, 10))

print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train(epoch):
    model.train()  # 신경망을 학습 모드로 전환
    # dropout이나 batchnorm처럼 train 과 test에서 다르게 동작하는 것들을 어떻게 해야하는지 알려줌
    # train에서는 dropout이 활성화 되고 batchnorm을 per-batch statistics를 사용
    
    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    for data, targets in loader_train:
        
        y_pred = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_fn(y_pred, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정

    print("epoch{}：완료\n".format(epoch))
    
def test():
    model.eval()  # 신경망을 추론 모드로 전환
    # test에서는 dropout이 비활성화되고 batchnorm은 running statistics 사용
    correct = 0
    
    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    with torch.no_grad():  # 추론 과정에서는 미분이 필요없음 (업데이트 안하기 때문)
        for data, targets in loader_test:
            outputs = model(data)
            
            _, predicted = torch.max(outputs.data, 1)  # 확률이 가장 높은 레이블이 무엇인지 계산
            correct += predicted.eq(targets.data.view_as(predicted)).sum()  # 정답과 일치한 경우 정답 카운트를 증가
    
    # 정확도 출력
    data_num = len(loader_test.dataset)  # 데이터 총 건수        
    print('\n테스트 데이터에서 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))        
test()
    
for epoch in range(3):
     train(epoch)
        
test()


# 하나 확인해보기
index = 2018

model.eval()  # 신경망을 추론 모드로 전환
data = X_test[index]
output = model(data)  # 데이터를 입력하고 출력을 계산
_, predicted = torch.max(output.data, 0)  # 확률이 가장 높은 레이블이 무엇인지 계산

print("예측 결과 : {}".format(predicted))

X_test_show = (X_test[index]).numpy()
plt.imshow(X_test_show.reshape(28, 28), cmap='gray')
print("이 이미지 데이터의 정답 레이블은 {:.0f}입니다".format(y_test[index]))

ImportError: DLL load failed: 지정된 모듈을 찾을 수 없습니다.